# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [2]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [3]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [4]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [5]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - 8f4bec14


In [6]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [7]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [8]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [10]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_2", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [12]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: /home/gpudja/AIE9/07_Deep_Agents/workspace


In [13]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
(empty directory)


In [14]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] sleep_notes.md (242 bytes)


In [15]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [16]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: /home/gpudja/AIE9/07_Deep_Agents/workspace


In [17]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Personalized Sleep Improvement Plan is Complete! ✅

I've created a comprehensive, science-based sleep improvement plan specifically tailored to address your three main challenges:

### **Key Highlights of Your Plan:**

1. **Gradual Schedule Consistency** - Instead of forcing an immediate change, you'll adjust your bedtime by just 15 minutes every 2-3 days until you reach a consistent 11 PM bedtime

2. **Phone Management Strategy** - A progressive approach starting with moving your charger out of the bedroom, then implementing a 1-hour screen cutoff, and eventually complete phone removal

3. **Morning Energy Focus** - Emphasizes consistent wake times (7 AM daily) and immediate morning light exposure to reset your circadian rhythm

4. **4-Week Implementation Plan** - Broken into manageable phases so you're not overwhelmed trying to change everything at once

### **Your First Steps (Start Tonight):**
- Move your phone charger to another room
- Set a consistent 7 AM

In [18]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_3] Research evidence-based sleep improvement strategies (completed)
✅ [todo_5] Research sleep hygiene best practices (completed)
✅ [todo_7] Save plan to file (completed)
✅ [todo_6] Research consistent sleep scheduling strategies (completed)
✅ [todo_8] Research screen time and blue light management (completed)
✅ [todo_10] Research sleep quality improvement methods (completed)
✅ [todo_12] Research gradual habit change approaches (completed)
✅ [todo_14] Compile comprehensive recommendations (completed)


Workspace contents:
  [FILE] personalized_sleep_improvement_plan.md (6686 bytes)
  [DIR] research/


---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
Explicit todo planning can slow things down for tasks I already know how to solve end-to-end, where the plan adds more ceremony than value. I've found todo items work best when they represent meaningful milestones rather than every micro-step, and if an agent creates todos but never completes or revisits them, the plan quickly becomes noise and stops being a reliable source of truth for progress.

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
I'd store the 16KB health document in the agent's workspace and only surface short, task-specific summaries when needed, because I don't want the prompt competing with static reference material. I'd keep long-term metrics in structured files and expose only rolling summaries to the prompt, while always keeping safety-critical information like allergies and medications in a fixed, validated prompt section, since treating those as optional context would be an unacceptable risk.

---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

#### 🏗️ Step 1: Create a research agent with appropriate tools

In this step, I'm creating a Deep Agent configured for research workflows:
- **Planning:** uses todo tools (`write_todos`, `update_todo`, `list_todos`)
- **Context Management:** uses a sandboxed filesystem backend pointing to `workspace/`
- **Goal:** the agent will later read `data/HealthWellnessGuide.txt`, create a research plan, and write a structured markdown report.


In [22]:
# Step 1: Create a research agent with appropriate tools
from pathlib import Path
from datetime import datetime

from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(root_dir=str(workspace_path), virtual_mode=True)

RUN_ID = datetime.now().strftime("%Y%m%d_%H%M")

research_tools = [write_todos, update_todo, list_todos]

research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=research_tools,
    backend=filesystem_backend,
    system_prompt="You are a Wellness Research Agent. Use todo lists to plan and track progress."
)

print("✅ Step 1 done")
print(f"📁 Workspace sandbox: {workspace_path}")
print(f"🆔 RUN_ID: {RUN_ID}")



✅ Step 1 done
📁 Workspace sandbox: /home/gpudja/AIE9/07_Deep_Agents/workspace
🆔 RUN_ID: 20260205_1243


#### 🏗️ Step 2: Add a tool to read from the data folder

Deep Agents file tools are sandboxed to the workspace directory.  
To let the agent use the provided reference material, I add a dedicated tool that reads `data/HealthWellnessGuide.txt` and returns its contents.



In [23]:
# Step 2: Add a tool to read from the data folder
from langchain_core.tools import tool

GUIDE_PATH = Path("data/HealthWellnessGuide.txt")

@tool
def read_wellness_guide() -> str:
    """Read the HealthWellnessGuide.txt reference document from the data folder."""
    if not GUIDE_PATH.exists():
        return f"Not found: {GUIDE_PATH}"
    return GUIDE_PATH.read_text(encoding="utf-8")

print("✅ Step 2 done: read_wellness_guide added")



✅ Step 2 done: read_wellness_guide added


#### 🏗️ Step 3: Create the agent with a research-focused system prompt

In this step, I create a research-oriented Deep Agent that:
- Plans work using todo lists
- Uses the provided wellness guide (`read_wellness_guide`) as a primary reference
- Writes a structured markdown report into the sandboxed `workspace/` directory
- Updates todo status as each research milestone completes


In [24]:
# Step 3: Create the agent with a research-focused system prompt
research_tools = [write_todos, update_todo, list_todos, read_wellness_guide]

research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=research_tools,
    backend=filesystem_backend,
    system_prompt=f"""
You are a Wellness Research Agent.

Workflow:
1) Create a todo list with 5–8 milestones for the research process.
2) Use the tool `read_wellness_guide` as a primary reference when relevant.
3) Produce a structured markdown report with at least 5 evidence-based strategies.
4) Save the report to: workspace/reports/stress_management_{RUN_ID}.md
5) Update todo statuses as you progress and finish with all todos completed.

Write practical guidance. For each strategy include: What it is, Why it works, How to apply it.
Return a short summary and confirm the report file path.
"""
)

print("✅ Step 3 done: research_agent recreated with guide tool + research prompt")



✅ Step 3 done: research_agent recreated with guide tool + research prompt


#### 🏗️ Step 4: Test with the stress management research task

This step runs the required test prompt to verify the agent:
- creates todos for the research process
- reads the provided wellness guide (`read_wellness_guide`)
- saves a structured markdown report into the sandboxed workspace
- updates todo statuses as tasks complete


In [ ]:
# Step 4: Test with the stress management research task
TODO_STORE.clear()

test_prompt = f"""
Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies.

MANDATORY REQUIREMENTS:
- First create a todo list for the research process (5-8 items)
- Use the tool `read_wellness_guide` to reference the provided guide
- Save the final report using the file tool to the EXACT RELATIVE PATH:
  reports/stress_management_{RUN_ID}.md
- Do NOT prefix the path with `workspace/` or `/workspace/`
- Update todo statuses as you work and finish with all todos marked as completed

After saving the report:
- List the contents of the `reports/` directory to confirm the file exists
"""

result = research_agent.invoke({
    "messages": [{"role": "user", "content": test_prompt}]
})

print("=== Agent final response ===")
print(result["messages"][-1].content)

print("\n=== Todos after run ===")
print(list_todos.invoke({}))

print("\n=== Reports directory contents ===")
reports_dir = Path("workspace/reports")
reports_dir.mkdir(parents=True, exist_ok=True)

for item in sorted(reports_dir.iterdir()):
    print(f"{'[DIR]' if item.is_dir() else '[FILE]'} {item.name}")

# Verify the expected report path
report_path = reports_dir / f"stress_management_{RUN_ID}.md"
print(f"\nExpected report path: {report_path}")
print("Report exists:", report_path.exists())

if report_path.exists():
    print("\n=== Report preview (first 40 lines) ===")
    lines = report_path.read_text(encoding="utf-8").splitlines()
    print("\n".join(lines[:40]))


=== Agent final response ===
## Summary

I have successfully completed the comprehensive stress management guide research and creation project. Here's what was accomplished:

### ✅ **All Requirements Met:**

1. **Created todo list**: 7 main milestones tracked throughout the process
2. **Used wellness guide**: Referenced the provided guide extensively for foundational information
3. **Developed 7+ evidence-based strategies**: Each includes "What it is," "Why it works," and "How to apply it"
4. **Saved to exact path**: `reports/stress_management_20260205_1243.md` (without workspace prefix)
5. **All todos completed**: Every milestone successfully finished

### 📋 **Guide Contents:**

The comprehensive guide includes **7 major evidence-based strategies:**

1. **Mindfulness and Meditation** - 23-35% cortisol reduction, neuroplasticity benefits
2. **Deep Breathing Exercises** - Immediate stress relief, vagus nerve activation
3. **Progressive Muscle Relaxation** - 30-45% cortisol reduction, bo

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [30]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [31]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [32]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
Perfect! I've successfully created your comprehensive morning routine guide for better energy. Here's what I've delivered:

## 🎯 **Project Complete!**

### **✅ What You Received:**

**📚 Comprehensive Research Foundation**
- Science-backed information on circadian rhythms and morning energy
- Evidence on exercise timing, nutrition strategies, and mindset practices
- Hormone regulation insights (cortisol, melatonin)
- Habit formation principles for sustainable routines

**📖 Complete Morning Energy Guide**
- **50+ page comprehensive guide** saved as `/comprehensive_morning_energy_guide.md`
- **4 Core Pillars**: Movement, Nutrition, Mindset, and Sleep Optimization
- **Quick-Start 15-minute routine** for immediate implementation
- **8-week implementation timeline** with success metrics
- **Lifestyle customizations** for parents, professionals, students, night shift workers, and more

**🛠️ Practical Implementation Tools**
- Daily checklists and tracking systems
- Soluti

In [34]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines (completed)
✅ [todo_3] Create comprehensive morning routine guide (completed)
✅ [todo_5] Save guide as formatted markdown file (completed)

Generated files in workspace:
  [FILE] additional_stress_management_techniques.md (8838 bytes)
  [FILE] comprehensive_morning_energy_guide.md (17348 bytes)
  [FILE] evidence_based_stress_management_guide.md (56281 bytes)
  [FILE] morning_energy_routine_guide.md (31749 bytes)
  [FILE] morning_routine_habit_formation_summary.md (2272 bytes)
  [FILE] morning_routine_research_summary.md (6284 bytes)
  [FILE] personalized_sleep_improvement_plan.md (6686 bytes)
  [DIR] reports/
  [DIR] research/
  [DIR] workspace/


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [35]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_alex"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Alex"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "morning",
    "communication_style": "detailed"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_alex
  name: {'value': 'Alex'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'morning', 'communication_style': 'detailed'}


In [36]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [37]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [38]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Hi Alex! I remember you - your primary goal is to improve energy levels with better sleep as a secondary goal. I also see you prefer morning exercise and detailed explanations, plus you're managing mild anxiety. Perfect timing to create an exercise routine that works for you!

Based on your profile, here's my personalized exercise recommendation:

## **Morning Energy-Boosting Routine** 🌅

**Frequency**: 5-6 days per week (with 1-2 rest days)
**Duration**: 30-45 minutes
**Best time**: Within 2 hours of waking up

### **Weekly Structure**:

**Monday, Wednesday, Friday - Energizing Cardio + Strength**
- 5-min dynamic warm-up (arm circles, leg swings, light stretching)
- 20-25 min moderate cardio (brisk walking, cycling, or swimming)
- 15 min bodyweight strength training (squats, push-ups, planks, lunges)
- 5-min cool-down stretches

**Tuesday, Thursday - Anxiety-Reducing Activities**
- 10-15 min yoga or tai chi (excellent for anxiety management)
- 20-25 min nature walk or 

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [39]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [40]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [41]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [42]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [43]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Personalized Wellness Assessment Complete! 

Based on your profile as a 35-year-old office worker looking to lose 15 pounds while managing sleep issues, I've created a comprehensive assessment that prioritizes your sleep quality as the foundation for success.

### 🎯 **Key Findings:**
- **Sleep is your #1 priority** - poor sleep directly sabotages weight loss and energy levels
- **Your vegetarian diet is a strength** - we just need to optimize it for your goals
- **Sedentary work life needs strategic intervention** - but it's totally manageable

### 🚀 **Your Action Plan:**

**Start TODAY:**
1. **Set up your sleep environment** (cool, dark, quiet bedroom)
2. **Set hourly movement reminders** (just 2-3 minutes each hour)
3. **Begin a simple food diary** to understand your current patterns

**This Week:**
- Establish a consistent sleep schedule (same bedtime/wake time)
- Add walking meetings or stair climbing to your work routine
- Plan protein-rich vegetarian meals

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [45]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli version: deepagents 0.0.18


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [46]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [47]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [48]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
## 🎉 Your Complete 2-Week Wellness Program is Ready!

Alex, I've created a comprehensive, personalized wellness program that addresses all three of your focus areas. Here's what you now have:

## 📋 **Program Summary**

**Duration:** 2 weeks (14 days)  
**Exercise:** 6 workouts, 30 minutes each (Mon/Wed/Fri)  
**Nutrition:** Complete vegetarian meal plans with energy & sleep focus  
**Mindfulness:** Daily stress management & sleep optimization techniques  

## 📁 **Your Reference Files**

I've created four detailed files for you to reference:

1. **`alex_complete_2week_wellness_program.md`** - Master overview with daily integration schedule
2. **`alex_exercise_plan.txt`** - Complete workout routines with progressions and anxiety-friendly modifications
3. **`alex_nutrition_plan.txt`** - Full 14-day vegetarian meal plan with shopping lists and prep guides
4. **`alex_mindfulness_plan.txt`** - Comprehensive stress management techniques and sleep optimization strategi

In [49]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Create 2-week exercise routine (completed)
✅ [todo_3] Develop vegetarian meal plan (completed)
✅ [todo_5] Design stress management and sleep optimization plan (completed)
✅ [todo_7] Create comprehensive program document (completed)
✅ [todo_9] Save individual plan files (completed)

GENERATED FILES
  [FILE] additional_stress_management_techniques.md (8838 bytes)
  [FILE] alex_complete_2week_wellness_program.md (6110 bytes)
  [FILE] alex_exercise_plan.txt (8368 bytes)
  [FILE] alex_mindfulness_plan.txt (23376 bytes)
  [FILE] alex_nutrition_plan.txt (19216 bytes)
  [FILE] comprehensive_morning_energy_guide.md (17348 bytes)
  [FILE] evidence_based_stress_management_guide.md (56281 bytes)
  [DIR] home/
  [FILE] meal_plan_alex_2_weeks.txt (5526 bytes)
  [FILE] morning_energy_routine_guide.md (31749 bytes)
  [FILE] morning_routine_habit_formation_summary.md (2272 bytes)
  [FILE] morning_routine_research_summary.md (6284 bytes)
  [FILE] personalized_sleep_improveme

In [50]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of morning_routine_habit_formation_summary.md:
# Research Summary: Understanding Habit Formation in Morning Routines

## Importance of Habit Formation
Habit formation is crucial for establishing and maintaining sustainable morning routines that have a significant impact on overall energy levels and productivity throughout the day. Understanding how habits are formed and what influences their consistency can enhance the effectiveness of morning rituals.

### Key Insights:
1. **Cue-Routine-Reward Loop**: According to Charles Duhigg's model, habits consist of a cue (trigger), routine (behavior), and reward (positive outcome). Identifying and reinforcing these elements can help in developing consistent morning habits.
   - **Actionable Tip**: Establish a clear cue for your morning routine, such as waking up at a specific time, followed by a specific sequence of activities, and rewarding yourself (e.g., enjoying a favorite coffee).
2. **Consistency is Key**: Research shows that co

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
When designing subagents, I keep shared tools only for truly shared resources (like the same filesystem backend), and give distinct tools when a specialist needs unique capabilities or I want stricter safety/side-effects (e.g., only the coordinator can update todos or write “final” files). I pick models by matching cost + risk (cheap/fast for bounded research or drafting; stronger models for integration and safety-sensitive guidance), and I aim for subagents that map to stable domains (exercise/nutrition/mindfulness) rather than micro-roles, otherwise orchestration overhead starts to dominate.

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
For a production wellness app, I'd start by adding basic safety guardrails so the agent knows what it can and cannot give advice on (for example, no medical diagnosis, clear disclaimers, and escalation when topics become sensitive). I'd also move from in-memory storage to a persistent database, isolate data per user, add simple monitoring to see what the agent is doing and where it fails, and control costs by using cheaper models for specialist subagents while keeping stronger models only for coordination and final responses.

---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

#### 🏗️ Step 1: Define specialized subagents

In this step, I define specialized subagents that my Wellness Coach can delegate to.
Each subagent focuses on a specific wellness domain and works in an isolated context
to keep the main agent efficient and well-structured.



In [ ]:
# Step 1: Define your subagent configurations
exercise_subagent = {
    "name": "exercise-specialist",
    "description": "Creates realistic workout plans and progression for a 30-day challenge.",
    "system_prompt": """You are an exercise coach.
Design safe, progressive workouts suitable for a 30-day wellness challenge.
Focus on consistency over intensity.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

nutrition_subagent = {
    "name": "nutrition-specialist",
    "description": "Provides simple, sustainable nutrition guidance for daily wellness challenges.",
    "system_prompt": """You are a nutrition coach.
Give practical meal and habit advice that is easy to follow daily.
Avoid extreme diets and focus on consistency.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

mindfulness_subagent = {
    "name": "mindfulness-specialist",
    "description": "Supports stress management, reflection, and sleep habits during the challenge.",
    "system_prompt": """You are a mindfulness coach.
Provide short daily practices for stress reduction, reflection, and sleep quality.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

print("✅ Step 1 complete: Subagents defined")

✅ Step 1 complete: Subagents defined


#### 🏗️ Step 2: Create additional tools

In this step, I create custom tools to support the 30-day wellness challenge.
These tools allow the agent to:
- Store daily check-in notes
- Generate and save weekly summary reports


In [54]:
# Step 2: Create additional tools
from datetime import date
from langchain_core.tools import tool

@tool
def save_daily_checkin(user_id: str, day: int, notes: str) -> str:
    """Save a daily check-in note for the wellness challenge."""
    path = f"checkins/{user_id}/day_{day}.md"
    content = f"# Day {day} Check-in ({date.today()})\n\n{notes}"
    return write_file.invoke({"path": path, "content": content})

@tool
def write_weekly_summary(user_id: str, week: int, summary: str) -> str:
    """Write a weekly summary report for the wellness challenge."""
    path = f"summaries/{user_id}/week_{week}_summary.md"
    content = f"# Week {week} Summary\n\n{summary}"
    return write_file.invoke({"path": path, "content": content})

@tool
def save_user_feedback(user_id: str, feedback: str) -> str:
    """Save user feedback to long-term memory to adapt future recommendations."""
    memory_store.put((user_id, "feedback"), f"feedback_{date.today()}", {"value": feedback})
    return "User feedback saved"

print("✅ Step 2 complete: Daily check-in , weekly summary, and user feedback tools created")

✅ Step 2 complete: Daily check-in , weekly summary, and user feedback tools created


#### 🏗️ Step 3: Build the main coordinator agent

In this step, I build the main Wellness Coach agent.
This agent coordinates the 30-day wellness challenge by:
- creating and updating todos
- delegating domain-specific work to subagents
- storing daily notes and weekly summaries
- remembering user preferences across interactions

In [ ]:
# Step 3: Build the main coordinator agent
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True
)

wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Context + tracking
        save_daily_checkin,
        write_weekly_summary,
        # Memory
        get_user_profile,
        save_user_preference,
        save_user_feedback,
        # Skills (optional, on-demand)
        load_skill,
    ],
    backend=filesystem_backend,
    subagents=[
        exercise_subagent,
        nutrition_subagent,
        mindfulness_subagent,
    ],
    system_prompt="""You are a Wellness Coach responsible for running a 30-day wellness challenge.

Workflow:
1. Always start by creating a todo list for the full 30-day challenge.
2. Review the user's profile and preferences before making recommendations.
3. Delegate:
   - exercise planning to exercise-specialist
   - nutrition guidance to nutrition-specialist
   - stress and sleep routines to mindfulness-specialist
4. Track daily progress using daily check-ins.
5. Adapt recommendations based on user feedback.
6. Save weekly summary reports to files.
7. Ensure all todos are completed by the end of the interaction.

Guidelines:
- Keep recommendations practical and sustainable.
- Respect all user preferences and constraints.
- Save all artifacts to the workspace for later reference.
"""
)

print("✅ Step 3 complete: Wellness Coach coordinator agent created")

✅ Step 3 complete: Wellness Coach coordinator agent created


#### 🏗️ Step 4: Test the 30-day wellness challenge

In this step, I test the complete Wellness Coach system by:
- creating a personalized 30-day challenge
- generating todos
- delegating work to subagents
- saving weekly summaries and daily tracking artifacts


In [ ]:
# Step 4: Test with a personalized 30-day challenge
TODO_STORE.clear()

user_id = "user_goran"

personalized_prompt = f"""
Hi! My user_id is {user_id}.

I want to start a 30-day wellness challenge.

Personal context:
- I enjoy hiking and spending time outdoors
- I have occasional knee pain, so exercises must be joint-friendly
- I wake up very early and my sleep quality is poor

Goals:
- Stay active without aggravating my knee
- Build strength and endurance to support hiking
- Improve sleep quality and reduce early awakenings

Constraints:
- 30 minutes per day on weekdays
- Prefer morning activity
- Vegetarian diet

Requirements:
- Create a todo list for the full 30-day challenge
- Delegate exercise planning, nutrition guidance, and sleep/stress strategies to specialists
- Adapt recommendations to knee pain and sleep issues
- Save daily check-ins and a weekly summary report
"""

result = wellness_coach.invoke({
    "messages": [{"role": "user", "content": personalized_prompt}]
})

print("=== Wellness Coach response ===")
print(result["messages"][-1].content)

print("\n=== Todos after challenge setup ===")
print(list_todos.invoke({}))


=== Wellness Coach response ===
## 🎉 Your 30-Day Wellness Challenge is Ready to Launch! 

Goran, I've successfully created your comprehensive wellness challenge program tailored specifically to your needs! Here's what you now have:

### ✅ **Completed Setup:**

1. **📋 Comprehensive Todo List** - 18 tasks to guide us through the full 30 days
2. **👤 Your Profile Saved** - All preferences stored for knee pain, sleep issues, hiking goals, and vegetarian diet
3. **💪 Joint-Friendly Exercise Plan** - 4-week progressive program from our exercise specialist
4. **🥗 Anti-Inflammatory Nutrition Guide** - Complete vegetarian meal planning from our nutrition specialist  
5. **😴 Sleep & Stress Management Plan** - Targeted strategies for early awakenings from our mindfulness specialist
6. **📅 Week 1 Action Plan** - Your detailed daily schedule starting tomorrow
7. **📝 Daily Check-in System** - Template and tracking system ready to use

### 🎯 **Your Personalized Program Highlights:**

**Exercise Focus:*

#### 🏗️ Step 5: Simulate a daily check-in and adaptation

In this step, I simulate a daily check-in during the 30-day challenge.
Based on the user's feedback, the Wellness Coach:
- stores the daily check-in
- saves user feedback to memory
- adapts future recommendations


In [57]:
# Step 5: Simulate a daily check-in and adaptation
user_id = "user_goran"
day = 3

daily_feedback = """
Today I completed the workout, but my knee felt slightly sore afterward.
The workout intensity was okay, but I'd prefer more mobility and less strength work.
I also woke up at 4:30 AM again and had trouble falling back asleep.
"""

# Save daily check-in to filesystem
checkin_result = save_daily_checkin.invoke({
    "user_id": user_id,
    "day": day,
    "notes": daily_feedback
})
print(checkin_result)

# Save feedback to long-term memory for adaptation
memory_result = save_user_feedback.invoke({
    "user_id": user_id,
    "feedback": "Knee soreness after workouts; reduce strength load and add mobility. Persistent early awakenings."
})
print(memory_result)

# Ask the coach to adapt recommendations based on feedback
adaptation_prompt = f"""
Hi, this is my check-in for Day {day}.

Please review my feedback and adapt the upcoming recommendations accordingly:
- Adjust exercise plan to reduce knee strain and add more mobility
- Update sleep strategy to address early morning awakenings
"""

result = wellness_coach.invoke({
    "messages": [{"role": "user", "content": adaptation_prompt}]
})

print("\n=== Adapted Coach Response ===")
print(result["messages"][-1].content)

print("\n=== Updated Todos ===")
print(list_todos.invoke({}))


Wrote 259 characters to checkins/user_goran/day_3.md
User feedback saved

=== Adapted Coach Response ===
Perfect! I've successfully adapted your wellness plan based on your Day 3 feedback. Here's what I've done:

## ✅ Day 3 Check-in Complete

### 🏃‍♂️ **Exercise Plan Adaptations** 
Your new knee-friendly exercise plan includes:
- **Low-impact alternatives** to high-stress movements
- **Daily mobility work** (10-15 minutes) focusing on hips, quads, hamstrings, and calves
- **Joint-friendly progressions** from seated exercises to modified squats and reverse lunges
- **Comprehensive warm-up and cool-down routines** for joint health
- Activities like swimming, cycling, and elliptical instead of high-impact options

### 😴 **Updated Sleep Strategy**
To address your early morning awakenings:
- **20-minute rule**: If awake for more than 20 minutes, get up and do quiet activities until sleepy
- **Gradual sleep schedule adjustment**: Shifting bedtime 15 minutes earlier every few days
- **Morning

#### 🏗️ Additional Step 6: Mid-Challenge Review and Program Adjustment

I perform a mid-challenge review to evaluate progress so far and adapt the
remaining wellness plan.

This step demonstrates:
- Long-horizon planning beyond daily check-ins
- Use of stored feedback and files
- Strategic adaptation instead of one-off reactions

The agent:
1. Reviews accumulated daily check-ins
2. Summarizes patterns and challenges
3. Updates upcoming recommendations
4. Adjusts remaining todos if needed


In [58]:
# Step 6: Mid-challenge review and adaptation (Day 14)
user_id = "user_goran"
review_day = 14

mid_review_prompt = f"""
We are at Day {review_day} of my 30-day wellness challenge.

Please perform a mid-challenge review by:
1. Reviewing my stored daily check-ins and feedback so far
2. Identifying key patterns (knee pain, sleep issues, motivation, energy)
3. Adjusting the remaining 16 days of the program accordingly:
   - Modify exercise volume/intensity if needed
   - Refine sleep and stress strategies
4. Updating the existing todos to reflect any changes
5. Writing a short mid-challenge summary report to:
   summaries/{user_id}/mid_challenge_review.md
"""

result = wellness_coach.invoke({
    "messages": [{"role": "user", "content": mid_review_prompt}]
})

print("=== Mid-Challenge Review Response ===")
print(result["messages"][-1].content)

print("\n=== Todos after mid-challenge review ===")
print(list_todos.invoke({}))


=== Mid-Challenge Review Response ===
Perfect! I've completed your comprehensive mid-challenge review for Day 14 of your 30-day wellness challenge. Here's what we've accomplished:

## ✅ **Mid-Challenge Review Complete**

### **Key Findings from Your First 14 Days:**
1. **Excellent consistency** with morning exercise routine
2. **Knee soreness** after strength workouts (Day 3 feedback)
3. **Sleep maintenance issues** - consistent 4:30 AM wake-ups
4. **Strong preference** for mobility over strength training

### **Major Program Adjustments Made:**

#### 🏃‍♂️ **Exercise Program (Days 15-30)**
- **Shifted focus:** 60% mobility/flexibility, 40% gentle strength
- **New weekly structure** with dedicated recovery days
- **Knee-friendly alternatives** for sensitive days
- **Enhanced joint care protocols** before, during, and after workouts

#### 😴 **Sleep Strategy Enhancements**
- **Earlier evening routine** starting at 8:30 PM
- **Specific 4:30 AM wake-up protocols** with step-by-step response

---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)